In [1]:
!pip install scikit-learn
!pip install numpy --upgrade
!pip install librosa --upgrade
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
astropy 5.3.4 requires numpy<2,>=1.21, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 2.0.0 which is incompatible.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.0.0 which is incompatible.
ibis-framework 8.0.0 requires numpy<2,>=1, but you have numpy 2.0.0 which is incompatible.
numba 0.58.1 requires numpy<1.27,>=1.22, but you have numpy 2.0.0 which is incompatible.
rmm-cu12 24.4.0 requires numpy<2.0a0,>=1.23, but you have numpy 2.0.0 which is incompatible.
scipy 1.11.4 requires numpy<1.28.0,>=1.21.6

In [1]:
import os
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

!unzip -uq "/content/drive/MyDrive/SR/task4.zip" -d "/content/files/"

In [4]:
def encoder_block(inputs, num_filters):

    # Convolution with 3x3 filter followed by ReLU activation
    x = layers.Conv2D(num_filters,
                               3,
                               padding = 'valid')(inputs)
    x = layers.Activation('relu')(x)

    # Convolution with 3x3 filter followed by ReLU activation
    x = layers.Conv2D(num_filters,
                               3,
                               padding = 'valid')(x)
    x = layers.Activation('relu')(x)

    # Max Pooling with 2x2 filter
    x = layers.MaxPool2D(pool_size = (2, 2),
                                  strides = 2)(x)

    return x

In [5]:
def decoder_block(inputs, skip_features, num_filters):

    # Upsampling with 2x2 filter
    x = layers.Conv2DTranspose(num_filters,
                                        (2, 2),
                                        strides = 2,
                                        padding = 'valid')(inputs)

    # Copy and crop the skip features
    # to match the shape of the upsampled input
    skip_features = tf.image.resize(skip_features,
                                    size = (x.shape[1],
                                            x.shape[2]))
    x = layers.Concatenate()([x, skip_features])

    # Convolution with 3x3 filter followed by ReLU activation
    x = layers.Conv2D(num_filters,
                               3,
                               padding = 'valid')(x)
    x = layers.Activation('relu')(x)

    # Convolution with 3x3 filter followed by ReLU activation
    x = layers.Conv2D(num_filters, 3, padding = 'valid')(x)
    x = layers.Activation('relu')(x)

    return x

In [25]:

# Unet code


def unet_model(input_shape = (572,572,1), num_classes = 1):
    inputs = layers.Input(input_shape)

    # Contracting Path
    s1 = encoder_block(inputs, 64)
    s2 = encoder_block(s1, 128)
    s3 = encoder_block(s1, 256)
    s4 = encoder_block(s3, 512)

    # Bottleneck
    b1 = layers.Conv2D(1024, 3, padding = 'valid')(s4)
    b1 = layers.Activation('relu')(b1)
    b1 = layers.Conv2D(1024, 3, padding = 'valid')(b1)
    b1 = layers.Activation('relu')(b1)

    # Expansive Path
    s5 = decoder_block(b1, s4, 512)
    s6 = decoder_block(s5, s3, 256)
    s7 = decoder_block(s6, s2, 128)
    s8 = decoder_block(s6, s1, 64)

    # Output
    outputs = layers.Conv2D(num_classes,
                                     1,
                                     padding = 'valid',
                                     activation = 'sigmoid')(s8)

    model = keras.models.Model(inputs = inputs,
                                  outputs = outputs,
                                  name = 'U-Net')
    return model

if __name__ == '__main__':
    model = unet_model(input_shape=(572, 572, 1), num_classes=2)
    model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 572, 572, 1)]        0         []                            
                                                                                                  
 conv2d_76 (Conv2D)          (None, 570, 570, 64)         640       ['input_5[0][0]']             
                                                                                                  
 activation_72 (Activation)  (None, 570, 570, 64)         0         ['conv2d_76[0][0]']           
                                                                                                  
 conv2d_77 (Conv2D)          (None, 568, 568, 64)         36928     ['activation_72[0][0]']       
                                                                                              

In [ ]:
# dim 512*512*1

In [7]:
root_dir='/content/files/membrane/'
X_train_dir=root_dir+'train/image/'
Y_train_dir=root_dir+'train/label/'
test_dir=root_dir+'test/'
train_dir=root_dir+'train/'

In [27]:
X_train = []
Y_train = []
X_test = []
Y_test = []

for i in range(30):
        train_img = Image.open((train_dir+"image/{index}.png").format(index=i))
        train_img = train_img.resize((484, 484))
        img_array = image.img_to_array(train_img)
        # img_array = np.expand_dims(img_array[:,:,:1], axis=0)
        img_array = img_array / 255.
        train_mask = Image.open((train_dir+"label/{index}.png").format(index=i))
        train_mask = train_mask.resize((484, 484))
        mask_array = image.img_to_array(train_mask)
        # mask_array = np.expand_dims(mask_array[:,:,:1], axis=0)
        mask_array = mask_array / 255.
        X_train.append(img_array)
        Y_train.append(mask_array)
        test_img=Image.open((test_dir+"{index}.png").format(index=i))
        test_img = test_img.resize((572, 572))
        img_array = image.img_to_array(test_img)
        # img_array = np.expand_dims(img_array[:,:,:1], axis=0)
        img_array = img_array / 255.
        test_mask=Image.open((test_dir+"{index}_predict.png").format(index=i))
        test_mask = test_mask.resize((484, 484))
        mask_array = image.img_to_array(test_mask)
        # mask_array = np.expand_dims(mask_array[:,:,:1], axis=0)
        mask_array = mask_array / 255.
        X_test.append(img_array)
        Y_test.append(mask_array)


X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
print(X_train.shape)
print(Y_train.shape)

(30, 484, 484, 1)
(30, 484, 484, 1)


In [12]:
from skimage.io import imread
from skimage.transform import resize

In [19]:

)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-3), loss="sparse_categorical_crossentropy",metrics=['accuracy']
)



epochs = 2
history=model.fit(
    X_train,Y_train,
    epochs=epochs,
    batch_size=8,
    validation_split=0.1
)

Epoch 1/2
1/1 [==============================] - 177s 177s/step - loss: 0.6171 - accuracy: 0.7596 - val_loss: 0.6108 - val_accuracy: 0.7301
Epoch 2/2
1/1 [==============================] - 168s 168s/step - loss: 0.5944 - accuracy: 0.7596 - val_loss: 0.5656 - val_accuracy: 0.7301


In [28]:
model = unet_model(input_shape=(572,572, 1), num_classes=2)
print(X_test[0].shape)
predictions = model.predict(X_test[0])

# Convert predictions to a numpy array and resize to original image size
predictions = np.squeeze(predictions, axis=0)
predictions = np.argmax(predictions, axis=-1)
predictions = Image.fromarray(np.uint8(predictions*255))
predictions = predictions.resize((572,572))

# Save the predicted image
predictions.save('predicted_image.jpg')
predictions

(572, 572, 1)


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'conv2d_114' (type Conv2D).
    
    Negative dimension size caused by subtracting 3 from 1 for '{{node U

In [35]:

# Load the image
img = Image.open(train_dir+"image/20.png")
# Preprocess the image
img = img.resize((572, 572))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array[:,:,:3], axis=0)
img_array = img_array / 255.

# Load the model
model = unet_model(input_shape=(572, 572, 1), num_classes=2)

# Make predictions
predictions = model.predict(img_array)

# Convert predictions to a numpy array and resize to original image size
predictions = np.squeeze(predictions, axis=0)
predictions = np.argmax(predictions, axis=-1)
predictions = Image.fromarray(np.uint8(predictions*255))
predictions = predictions.resize((img.width, img.height))



predictions.save('predicted_image.jpg')


1/1 [==============================] - 0s 197ms/step
